In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [25]:
design.get_chip_z('Q_chip')

'7um'

In [3]:
design.chips['C_chip'].size['size_x']='6mm'
design.chips['C_chip'].size['size_y']='6mm'
design.chips['Q_chip'].size['size_x']='4mm'
design.chips['Q_chip'].size['size_y']='4mm'
design.chips['Q_chip'].size['center_z']='7um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_4.append(a)
print(resonator_length_4)


[3.876627341935142, 3.7623941699207104, 3.654173270104565]


# Flipchip

In [ ]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-1300um',
                                            pos_y='-470um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="50um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [ ]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-1700um',
                                         pos_y = '1300um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '1700um',
                                         pos_y = '1300um',
                                         orientation='180',
                                         lead_length = '30um'))

In [ ]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='-0.585mm',
                                                pos_y='0mm',
                                                orientation='270'))

In [ ]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [ ]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [ ]:
jogs1=OrderedDict()
jogs1[0]=['R','350um']
jogs1[1]=['R', '400um']
jogs1[2]=['L','515um']


In [ ]:
rr0=connect('rr0', 'c0', 'in','stg', 'short', 
            resonator_length_4[1],jogs1,jogs, 
            '400um','0um', '1388um', '100um')

# EPR Analysis

In [35]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [36]:
hfss.start()

INFO 06:46PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:46PM [load_ansys_project]: 	Opened Ansys App
INFO 06:46PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 06:46PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project2
INFO 06:46PM [connect_design]: No active design found (or error getting active design).
INFO 06:46PM [connect]: 	 Connected to project "Project2". No design detected


True

In [37]:
hfss.activate_ansys_design("flip7um_V2", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

06:46PM 02s WARNING [activate_ansys_design]: The design_name=flip7um_V2 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 06:46PM [connect_design]: 	Opened active design
	Design:    flip7um_V2 [Solution type: Eigenmode]
WARNING 06:46PM [connect_setup]: 	No design setup detected.
WARNING 06:46PM [connect_setup]: 	Creating eigenmode default setup.
INFO 06:46PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [23]:
hfss.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [38]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [39]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.09',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.09',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [41]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 09:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpgzou10m_.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 1, False
INFO 09:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp03_sqlp5.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 1, False
INFO 09:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpz7qc_avs.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 2, False
INFO 09:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpu6blf47n.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 3, False
INFO 09:19PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp20aaudq4.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5090000000, Maxwell, 4, False
INFO 09:19

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,375.960,-120.97000,-117.9500,-74.30800
pad_bot_Q0,-120.970,150.88000,-27.9020,-0.35029
pad_top_Q0,-117.950,-27.90200,151.0900,-3.48910
quad_coupler_c0,-74.308,-0.35029,-3.4891,78.47100


In [42]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.09
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.817639 [GHz]
EC 213.818702 [MHz]
EJ 14.854145 [GHz]
alpha -239.412526 [MHz]
dispersion 0.365557 [KHz]
Lq 10.995564 [nH]
Cq 90.591833 [fF]
T1 4932.090394 [us]

**Coupling Properties**

tCqbus1 -1.568465 [fF]
gbus1_in_MHz -18.945683 [MHz]
χ_bus1 -0.031349 [MHz]
1/T1bus1 32.269267 [Hz]
T1bus1 4932.090394 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.605858,294.545888,14.854145,-338.045476,10.755351,[-42.84946133029578],[-0.4669276202984607],0.466928,42.849461
2,5.18468,249.611461,14.854145,-282.630318,2.010227,[-22.442467290005755],[-0.07022844584855518],0.070228,22.442467
3,4.981696,229.43588,14.854145,-258.168711,0.8082,[-15.79620895355234],[-0.02672629545833401],0.026726,15.796209
4,4.866013,218.360063,14.854145,-244.850584,0.464518,[-17.457253983356587],[-0.028246394512573846],0.028246,17.457254
5,4.83651,215.584012,14.854145,-241.524828,0.401602,[-18.521239148364526],[-0.03066057136177364],0.030661,18.521239
6,4.83269,215.226064,14.854145,-241.09636,0.394054,[-18.190919504679872],[-0.029438440977456216],0.029438,18.190920
7,4.817639,213.818702,14.854145,-239.412526,0.365557,[-18.94568268995035],[-0.031348981294658844],0.031349,18.945683


In [43]:
c1.sim.close()

Warning! 6 COM references still alive
Ansys will likely refuse to shut down
